# Benchmarks

## 🎯 Number of calls and results benchmarks

## ⚡ Speed and memory consumption benchmark

- import some functions from stochopy to test the code

False

In [1]:
# python lbfgsb works with numpy arrays
import numpy as np
from lbfgsb import minimize_lbfgsb
from scipy.optimize import minimize
import logging
from lbfgsb.benchmarks import (
    ackley,
    ackley_grad,
    beale,
    beale_grad,
    griewank,
    griewank_grad,
    quartic,
    quartic_grad,
    rastrigin,
    rastrigin_grad,
    rosenbrock,
    rosenbrock_grad,
    sphere,
    sphere_grad,
    styblinski_tang,
    styblinski_tang_grad,
)

import matplotlib.pyplot as plt

In [2]:
from typing import Callable
from lbfgsb.types import NDArrayFloat


def plot_2d(lb: NDArrayFloat, ub: NDArrayFloat, f: Callable) -> None:
    x = np.linspace(lb, ub)
    plt.contourf()

- Define a logger for the display

In [3]:
logging.info("Test")
main_logger = logging.getLogger(" -MAIN- ")
main_logger.setLevel(level=logging.INFO)
main_logger.info("test the main logger")

solver_logger = logging.getLogger("L-BFGS-B")
solver_logger.setLevel(level=logging.INFO)
solver_logger.info("test the solver logger")

INFO: -MAIN- :test the main logger
INFO:L-BFGS-B:test the solver logger


In [4]:
iprint = -1

- First example: min x^{\mathrm{T}}x such that x>=1. Optimal solution hits the boundary.

In [5]:
def quad(x):
    return 10 * x[0] ** 2 + x[1] ** 2


def grad_quad(x):
    return np.array([20 * x[0], 2 * x[1]])


ftol = 1e-5
gtol = 1e-5
l = np.array([1.0, 1.0])
u = np.array([np.inf, np.inf])
bounds = np.array((l, u)).T
x0 = np.array([5, 5])

In [6]:
bounds[0]

array([ 1., inf])

### New implementation

In [7]:
main_logger.info("====================== Quadratic example ======================")
opt_quad = minimize_lbfgsb(
    x0=x0,
    fun=quad,
    jac=grad_quad,
    bounds=bounds,
    ftol=ftol,
    gtol=gtol,
    iprint=iprint,
    logger=solver_logger,
)
main_logger.info("")
main_logger.info("")
xOpt = np.array([1, 1])
theoOpt = {"x": xOpt, "f": quad(xOpt), "df": grad_quad(xOpt)}
main_logger.info("Theoretical optimal value: ")
main_logger.info(theoOpt)
main_logger.info("Optimal value found: ")
main_logger.info(opt_quad)

INFO: -MAIN- :====================== Quadratic example ======================
INFO: -MAIN- :
INFO: -MAIN- :
INFO: -MAIN- :Theoretical optimal value: 
INFO: -MAIN- :{'x': array([1, 1]), 'f': 11, 'df': array([20,  2])}
INFO: -MAIN- :Optimal value found: 
INFO: -MAIN- :  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 1
      fun: 11.0
        x: [ 1.000e+00  1.000e+00]
      nit: 2
      jac: [ 2.000e+01  2.000e+00]
     nfev: 3
     njev: 3
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>


### Scipy

In [8]:
minimize(
    quad,
    x0,
    jac=grad_quad,
    bounds=bounds,
    method="l-bfgs-b",
    options={"gtol": gtol, "ftol": ftol, "disp": iprint, "iprint": iprint},
)

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 11.0
        x: [ 1.000e+00  1.000e+00]
      nit: 2
      jac: [ 2.000e+01  2.000e+00]
     nfev: 3
     njev: 3
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

# Second example: Ackley function

In [9]:
l = np.array([-2, -2, -1])
u = np.array([2, 2, 3])
bounds = np.array((l, u)).T
# x0 = np.array([0.12, 0.12])
x0 = np.array([-0.8, -1, 0.1])

In [10]:
opt_akley = minimize_lbfgsb(
    x0=x0,
    fun=ackley,
    jac=ackley_grad,
    bounds=bounds,
    maxcor=5,
    ftol=ftol,
    gtol=gtol,
    iprint=iprint,
    ftol_linesearch=1e-3,
    gtol_linesearch=0.9,
    xtol_linesearch=0.1,
    logger=solver_logger,
)
main_logger.info("")
main_logger.info("")
theoOpt = {"x": np.array([1, 1, 2]), "f": 0, "df": ackley_grad(np.array([1, 1, 3]))}
main_logger.info("Theoretical optimal value: ")
main_logger.info(opt_akley)
main_logger.info("Optimal value found: ")

/home/acollet/workspace/lbfgsb/lbfgsb/linesearch.py:247: UserWarning: WARNING: dcsrch did not converge within max iterations
  warnings.warn(task.decode("utf-8"))
INFO: -MAIN- :
INFO: -MAIN- :
INFO: -MAIN- :Theoretical optimal value: 
INFO: -MAIN- :  message: ABNORMAL_TERMINATION_IN_LNSRCH
  success: False
   status: 2
      fun: 3.4493521599753656
        x: [-8.000e-01 -1.000e+00  1.000e-01]
      nit: 0
      jac: [-3.232e+00 -1.550e+00 -1.076e+00]
     nfev: 21
     njev: 21
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
INFO: -MAIN- :Optimal value found: 


In [11]:
minimize(
    x0=x0,
    fun=ackley,
    jac=ackley_grad,
    bounds=bounds,
    method="l-bfgs-b",
    options=dict(
        maxcor=5,
        ftol=ftol,
        gtol=gtol,
        iprint=iprint,
    ),
)

  message: ABNORMAL_TERMINATION_IN_LNSRCH
  success: False
   status: 2
      fun: 3.4493521599753656
        x: [-8.000e-01 -1.000e+00  1.000e-01]
      nit: 0
      jac: [-3.232e+00 -1.550e+00 -1.076e+00]
     nfev: 21
     njev: 21
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>

## Second example : min Rosenbrock function

In [12]:
l = np.array([-2, -2])
u = np.array([2, 2])
bounds = np.array((l, u)).T
# x0 = np.array([0.12, 0.12])
x0 = np.array([-0.8, -1])

### New implementation

In [13]:
opt_rosenbrock = minimize_lbfgsb(
    x0=x0,
    fun=rosenbrock,
    jac=rosenbrock_grad,
    bounds=bounds,
    maxcor=5,
    ftol=ftol,
    gtol=gtol,
    iprint=iprint,
    ftol_linesearch=1e-3,
    gtol_linesearch=0.9,
    xtol_linesearch=0.1,
    logger=solver_logger,
)
main_logger.info("")
main_logger.info("")
theoOpt = {"x": np.array([1, 1]), "f": 0, "df": rosenbrock_grad(np.array([1, 1]))}
main_logger.info("Theoretical optimal value: ")
main_logger.info(theoOpt)
main_logger.info("Optimal value found: ")

INFO: -MAIN- :
INFO: -MAIN- :
INFO: -MAIN- :Theoretical optimal value: 
INFO: -MAIN- :{'x': array([1, 1]), 'f': 0, 'df': array([0., 0.])}
INFO: -MAIN- :Optimal value found: 


In [14]:
main_logger.info(opt_rosenbrock)

INFO: -MAIN- :  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FTOL
  success: True
   status: 0
      fun: 3.8738696318949315e-07
        x: [ 9.994e-01  9.988e-01]
      nit: 19
      jac: [-1.603e-03  1.794e-04]
     nfev: 24
     njev: 24
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>


In [15]:
rosenbrock_grad(opt_rosenbrock.x)

array([-0.00160319,  0.00017937])

### Scipy

In [16]:
opt_rosenbrock_sp = minimize(
    rosenbrock,
    x0,
    jac=rosenbrock_grad,
    bounds=bounds,
    method="l-bfgs-b",
    options={"gtol": gtol, "ftol": ftol, "iprint": iprint, "maxcor": 5},
)

In [17]:
opt_rosenbrock_sp

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 3.87386963183571e-07
        x: [ 9.994e-01  9.988e-01]
      nit: 19
      jac: [-1.603e-03  1.794e-04]
     nfev: 24
     njev: 24
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

## Third example : min Beale function

In [18]:
def beale2(x):
    return (
        (1.5 - x[0] + x[0] * x[1]) ** 2
        + (2.25 - x[0] + x[0] * x[1] ** 2) ** 2
        + (2.625 - x[0] + x[0] * x[1] ** 3) ** 2
    )


def beale_grad2(x):
    y1 = x[1]
    y2 = y1 * y1
    y3 = y2 * y1
    f1 = 1.5 - x[0] + x[0] * y1
    f2 = 2.25 - x[0] + x[0] * y2
    f3 = 2.625 - x[0] + x[0] * y3

    return np.array(
        [
            2 * (y1 - 1) * f1 + 2 * (y2 - 1) * f2 + 2 * (y3 - 1) * f3,
            2 * x[0] * f1 + 4 * x[0] * y1 * f2 + 6 * x[0] * y2 * f3,
        ]
    )


ftol = 1e-14
gtol = 1e-10
l = -4.5 * np.ones(2)
u = -l
bounds = np.array((l, u)).T
x0 = np.array([2.5, -1.3])

### New implementation

In [19]:
opt_beale = minimize_lbfgsb(
    x0=x0,
    fun=beale,
    jac=beale_grad,
    bounds=bounds,
    ftol=ftol,
    gtol=gtol,
    logger=solver_logger,
    iprint=iprint,
)
main_logger.info("")
main_logger.info("")
theoOpt = {
    "x": np.array([3, 0.5]),
    "f": beale(np.array([3, 0.5])),
    "df": beale_grad(np.array([3, 0.5])),
}
main_logger.info("Theoretical optimal value: ")
main_logger.info(theoOpt)
main_logger.info("Optimal value found: ")

INFO: -MAIN- :
INFO: -MAIN- :
INFO: -MAIN- :Theoretical optimal value: 
INFO: -MAIN- :{'x': array([3. , 0.5]), 'f': 0.0, 'df': array([0., 0.])}
INFO: -MAIN- :Optimal value found: 


In [20]:
main_logger.info(opt_beale)

INFO: -MAIN- :  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FTOL
  success: True
   status: 0
      fun: 1.4337068336583181e-21
        x: [ 3.000e+00  5.000e-01]
      nit: 16
      jac: [-7.089e-11  1.797e-10]
     nfev: 20
     njev: 20
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>


### Scipy

In [21]:
opt_beale_sp = minimize(
    beale,
    x0,
    jac=beale_grad,
    bounds=bounds,
    method="l-bfgs-b",
    options={"gtol": gtol, "ftol": ftol, "iprint": iprint},
)

In [22]:
opt_beale_sp

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 1.4337068336583181e-21
        x: [ 3.000e+00  5.000e-01]
      nit: 16
      jac: [-7.089e-11  1.797e-10]
     nfev: 20
     njev: 20
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

- Comparision of convergence